In [1]:
import dataset

ds = dataset.Dataset()

Using TensorFlow backend.


In [2]:
import numpy as np
def PosNormal(mean, sigma):
    x = np.random.normal(xbar,delta_xbar,1)
    return(x if x>=0 else PosNormal(mean,sigma))

In [3]:
from scipy.stats import truncnorm

def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

test = get_truncated_normal()

test.rvs((32, 32, 3))

array([[[ 0.21166043,  0.4772091 ,  0.80797618],
        [ 0.46942654,  0.45322036,  0.02051879],
        [ 0.95220257,  1.18126901,  0.35664092],
        ..., 
        [ 0.0083142 ,  1.39145741,  0.39999356],
        [ 0.09551466,  1.56019331,  1.45370658],
        [ 1.05749392,  0.01816132,  0.40965545]],

       [[ 1.31119971,  1.73755249,  0.92241258],
        [ 0.0947313 ,  2.44403379,  0.50431488],
        [ 0.83738252,  0.74038621,  0.98667467],
        ..., 
        [ 2.39383574,  1.39976457,  1.32206952],
        [ 0.56215961,  0.33298063,  0.224076  ],
        [ 0.10532339,  0.65301833,  1.92445925]],

       [[ 2.67766526,  0.5315205 ,  2.0223938 ],
        [ 0.54596307,  0.34105275,  0.1683546 ],
        [ 0.2733254 ,  0.16190436,  1.78073114],
        ..., 
        [ 0.9812969 ,  0.16630902,  1.43307844],
        [ 1.2388349 ,  0.53986013,  0.48368109],
        [ 1.54138333,  0.76396529,  0.5651463 ]],

       ..., 
       [[ 0.43545057,  0.05509278,  1.09501319],
        

In [4]:
def binary_crossentropy(y, p):
    return -1*(y*np.log(p) + (1-y)*np.log(1-p))

In [27]:
import numpy as np
from tqdm import tqdm
from IPython.core.debugger import set_trace
from copy import deepcopy

def generate_training_data(num_samples=10000, input_shape=(32, 32, 3)):
    np.random.seed(0)
    orig = np.random.uniform(size=(32, 32, 3))
    np.random.seed(1)
    target = np.random.uniform(size = (32, 32, 3))
    np.random.seed()
    
    inputs = []
    targets = []
    
    for i in tqdm(range(num_samples)):
        noise = (np.random.uniform(size = (32, 32, 3))-.5)*.5
        inputs.append([deepcopy(orig), deepcopy(noise)])
        orig_loss = binary_crossentropy(target, orig)
        orig_noise = orig+noise
        orig_noise = np.where(orig_noise < 0, 0, orig_noise)
        orig_noise = np.where(orig_noise > 1, 1, orig_noise)
        orig_noise_loss = binary_crossentropy(target, orig_noise)
        optimal_mask = np.where(orig_loss > orig_noise_loss, 1, 0)
        orig += (noise * optimal_mask)
        orig = np.where(orig < 0, 0, orig)
        orig = np.where(orig > 1, 1, orig)
        targets.append(deepcopy(orig))
    return np.array(inputs), np.array(targets)

inputs, targets = generate_training_data()

  0%|          | 0/10000 [00:00<?, ?it/s]/home/jprothero/Projects/miniconda3/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
100%|██████████| 10000/10000 [00:12<00:00, 775.80it/s]


In [28]:
from keras.layers import Conv2D, Dense, Input, LSTM, TimeDistributed, Reshape, ConvLSTM2D
from keras.models import Model

def create_random_lstm():
    inp = Input(shape = (2, 32, 32, 3))
    x = ConvLSTM2D(128, 1, return_sequences=True)(inp)
    x = ConvLSTM2D(128, 1, return_sequences=True)(x)
    out = ConvLSTM2D(3, 1, return_sequences=False)(x)

    model = Model(inputs=inp, outputs=out)
    model.summary()

    return model

model = create_random_lstm()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2, 32, 32, 3)      0         
_________________________________________________________________
conv_lst_m2d_3 (ConvLSTM2D)  (None, 2, 32, 32, 128)    67584     
_________________________________________________________________
conv_lst_m2d_4 (ConvLSTM2D)  (None, 2, 32, 32, 128)    131584    
_________________________________________________________________
conv_lst_m2d_5 (ConvLSTM2D)  (None, 32, 32, 3)         1584      
Total params: 200,752
Trainable params: 200,752
Non-trainable params: 0
_________________________________________________________________


In [30]:
from keras.optimizers import SGD, Nadam
from clr_callback import CyclicLR

def fit(model, inputs=inputs, targets=targets):
    optim = Nadam()
    base_lr = 0.001
    max_lr = 0.006
    clr = CyclicLR(base_lr=base_lr, max_lr=max_lr,
                   step_size=2000., mode='triangular')

    model.compile(optimizer=optim,
                  loss="binary_crossentropy")
    
    model.fit(inputs, targets, batch_size=32, callbacks=[clr])

In [31]:
fit(model)

Epoch 1/1
   64/10000 [..............................] - ETA: 3524s - loss: 5.2338

KeyboardInterrupt: 

In [ ]:
# assuming this worked perfectly, I would use this output mask to ignore certain parameter updates and probably
# scale the learning rate based on the confidence level
# could I do this internally though? 
# maybe I could learn to create random uniform numbers and from that random uniform number I can
# add and divide it to whatever the current number is, with the goal of something